In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import tensorflow as tf
tf.keras.backend.clear_session()
import pickle

In [22]:
df = pd.read_csv("data/fire.csv")
X = df.drop('Fire Alarm', axis=1)
y = df['Fire Alarm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
knn_model = KNeighborsClassifier(3)
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [24]:
with open('knn.pkl', 'wb') as pickle_out:
    pickle.dump(knn_model, pickle_out)

In [25]:
with open('knn.pkl', 'rb') as pickle_in:
    unpickled_list = pickle.load(pickle_in)

In [26]:
rf = RandomForestClassifier(random_state=42).fit(X_train, y_train)

In [27]:
with open('rf.pkl', 'wb') as pickle_out:
    pickle.dump(rf, pickle_out)

In [28]:
base_classifiers = [
    ('logistic', LogisticRegression()),
    ('random_forest', RandomForestClassifier(n_estimators=100)),
    ('knn', KNeighborsClassifier(3))
]

meta_classifier = DecisionTreeClassifier()
stacking_classifier = StackingClassifier(estimators=base_classifiers, final_estimator=meta_classifier)
stacking_classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

StackingClassifier(estimators=[('logistic', LogisticRegression()),
                               ('random_forest', RandomForestClassifier()),
                               ('knn', KNeighborsClassifier(n_neighbors=3))],
                   final_estimator=DecisionTreeClassifier())

In [29]:
with open('stacking_classifier.pkl', 'wb') as pickle_out:
    pickle.dump(stacking_classifier, pickle_out)

In [30]:
base_classifier = DecisionTreeClassifier()
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=100, random_state=42)
bagging_classifier.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100,
                  random_state=42)

In [31]:
with open('bagging_classifier.pkl', 'wb') as pickle_out:
    pickle.dump(bagging_classifier, pickle_out)

In [32]:
from sklearn.cluster import KMeans

In [33]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
kmeans.fit(X_train)
predictions = kmeans.predict(X_test)

In [34]:
with open('kmeans.pkl', 'wb') as pickle_out:
    pickle.dump(kmeans, pickle_out)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X[:30000], y[:30000], test_size=0.2, random_state=42)

In [36]:
model_cl = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(32, activation="relu", input_shape=(7,)),
        tf.keras.layers.Dense(16, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)

In [37]:
model_cl.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss="sparse_categorical_crossentropy")

In [38]:
model_cl.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                256       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_3 (Dense)             (None, 2)                 18        
                                                                 
Total params: 938 (3.66 KB)
Trainable params: 938 (3.66 

In [41]:
model_cl.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node sparse_categorical_crossentropy/Cast defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/loca

In [ ]:
y_pred_prob = model_cl.predict(X_classification_test)
y_pred = tf.argmax(y_pred_prob, axis=1)